In [1]:
import numpy as np

name = 'real_most'
# === Paths ===
input_path = f"{name}.npz"
output_path = f"{name}_with_scores.npz"

# === Load data ===
data = np.load(input_path)
coords = data["coords"]  # [T, N, 3]
visibs = np.clip(data["visibs"][..., 0], 0.0, 1.0)  # [T, N]

# === Compute average displacement magnitude per point ===
disp = np.linalg.norm(np.diff(coords, axis=0), axis=-1)  # [T-1, N]
avg_disp = np.mean(disp, axis=0)  # [N]

# === Normalize motion range ===
zero_motion_thresh = np.percentile(avg_disp, 10)
max_motion_thresh = np.percentile(avg_disp, 95)
disp_clipped = np.clip(
    (avg_disp - zero_motion_thresh) / (max_motion_thresh - zero_motion_thresh + 1e-8),
    0, 1
)

# === Compute higher-mean score (gentler exponential decay) ===
# - smaller k makes it flatter
# - increase base offset to shift average upward
k = 0.5  # gentler decay
base = 0.3  # raises the minimum contribution
score_per_point = 0.1 + 0.9 * (base + (1 - base) * np.exp(-k * disp_clipped))
score_per_point = np.clip(score_per_point, 0.1, 1.0)

# === Broadcast same score to all frames ===
coords_score = np.tile(score_per_point[None, :, None], (coords.shape[0], 1, 1))  # [T, N, 1]

# === Apply visibility weighting ===
coords_score = 0.1 + (coords_score - 0.1) * visibs[..., None]
coords_score = np.clip(coords_score, 0.1, 1.0)

# === Save updated .npz ===
np.savez(
    output_path,
    **{key: data[key] for key in data.files},
    coords_score=coords_score
)

# === Print stats ===
print(f"✅ coords_score saved to: {output_path}")
print(f"Shape: {coords_score.shape}")
print(f"Min: {coords_score.min():.4f}, Max: {coords_score.max():.4f}, Mean: {coords_score.mean():.4f}")


✅ coords_score saved to: real_most_with_scores.npz
Shape: (76, 100, 1)
Min: 0.2304, Max: 1.0000, Mean: 0.5265


In [2]:
import numpy as np

path = f"{name}_with_scores.npz"

# --- Load file ---
data = np.load(path)

# --- Check keys ---
print("Keys in file:", data.files)

# --- Read coord_score ---
if "coords_score" not in data.files:
    raise KeyError("coords_score not found in the file!")

coords_score = data["coords_score"] 

# --- Print stats ---
print("\ncoords_score summary:")
print("Shape:", coords_score.shape)
print("Min:", np.min(coords_score))
print("Max:", np.max(coords_score))
print("Mean:", np.mean(coords_score))
print("Median:", np.median(coords_score))
print("Std Dev:", np.std(coords_score))

# --- Print few sample values ---
print("\nSample scores (first 5 frames, first 5 points):")
print(np.round(coords_score[:5, :5, 0], 3))


Keys in file: ['coords', 'extrinsics', 'intrinsics', 'depths', 'video', 'visibs', 'unc_metric', 'coords_score']

coords_score summary:
Shape: (76, 100, 1)
Min: 0.23043734
Max: 1.0
Mean: 0.52650595
Median: 0.27125406
Std Dev: 0.3235102

Sample scores (first 5 frames, first 5 points):
[[0.752 0.752 0.853 0.869 0.837]
 [0.752 0.752 0.853 0.869 0.837]
 [0.752 0.752 0.853 0.869 0.837]
 [0.752 0.752 0.853 0.869 0.837]
 [0.752 0.752 0.853 0.869 0.837]]
